# Import packages and data preprocessing

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import numpy as np

# Connect with Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Load return data from crsp database
file_path = "/content/drive/My Drive/Thesis/mywork/Copy of crsp.csv"
crsp = pd.read_csv(file_path)
crsp

,permno,date,prc,ret,vol,ask,bid
0,10001,2014-07-01,10.40000,-0.013283,63701.0,10.44000,10.40000
1,10025,2014-07-01,36.95000,0.059650,47041.0,37.02000,36.79000
2,10026,2014-07-01,95.03000,0.009668,76584.0,95.11000,95.03000
3,10028,2014-07-01,1.52000,0.041096,48495.0,1.54000,1.50000
4,10032,2014-07-01,44.16000,0.020097,151414.0,44.20000,44.16000
...,...,...,...,...,...,...,...
21449072,93426,2024-12-31,23.47000,0.010332,62710.0,23.53000,23.47000
21449073,93427,2024-12-31,219.88000,0.023841,416107.0,220.10001,219.74001
21449074,93429,2024-12-31,195.39999,-0.002705,471701.0,195.39999,195.38000
21449075,93434,2024-12-31,7.99000,-0.052195,12585.0,8.01000,7.73000


In [ ]:
crsp_test = crsp[(crsp["date"]>= "2015-01-01" ) & (crsp["date"]<= "2024-12-31")]
crsp_test

,permno,date,prc,ret,vol,ask,bid
909069,10001,2015-01-02,11.02000,0.000000,8644.0,10.93000,10.90000
909070,10025,2015-01-02,58.00000,-0.002580,32084.0,58.32000,57.82000
909071,10026,2015-01-02,107.69000,-0.009929,88837.0,107.69000,107.45000
909072,10028,2015-01-02,1.27000,0.040984,1500.0,1.26000,1.22000
909073,10032,2015-01-02,40.57000,-0.015530,163298.0,40.57000,40.53000
...,...,...,...,...,...,...,...
21449072,93426,2024-12-31,23.47000,0.010332,62710.0,23.53000,23.47000
21449073,93427,2024-12-31,219.88000,0.023841,416107.0,220.10001,219.74001
21449074,93429,2024-12-31,195.39999,-0.002705,471701.0,195.39999,195.38000
21449075,93434,2024-12-31,7.99000,-0.052195,12585.0,8.01000,7.73000


In [ ]:
# Load link table
file_path = "/content/drive/My Drive/Thesis/mywork/Copy of link_table.csv"
link_table = pd.read_csv(file_path)
link_table

,gvkey,permno,permco,linktype,linkdt,linkenddt
0,1000,25881.0,23369.0,LU,1970-11-13,1978-06-30
1,1001,10015.0,6398.0,LU,1983-09-20,1986-07-31
2,1002,10023.0,22159.0,LC,1972-12-14,1973-06-05
3,1003,10031.0,6672.0,LU,1983-12-07,1989-08-16
4,1004,54594.0,20000.0,LU,1972-04-24,NaN
...,...,...,...,...,...,...
39325,353444,23209.0,59330.0,LC,2022-07-22,NaN
39326,354003,23698.0,37493.0,LS,2023-02-24,NaN
39327,355398,25134.0,59886.0,LC,2024-05-17,NaN
39328,356128,24704.0,59765.0,LC,2024-01-19,NaN


In [ ]:
df_test_merge = pd.merge(crsp_test, link_table, on='permno', how='inner')

# clern merged data
df_test_merge['linkdt'] = pd.to_datetime(df_test_merge['linkdt'])
df_test_merge['linkenddt'] = pd.to_datetime(df_test_merge['linkenddt'])
df_test_merge  = df_test_merge [(df_test_merge['date'] >= df_test_merge['linkdt']) | (df_test_merge['linkdt'].isnull())]
df_test_merge  = df_test_merge [(df_test_merge['date'] <= df_test_merge['linkenddt']) | (df_test_merge['linkenddt'].isnull())]

df_test_merge

,permno,date,prc,ret,vol,ask,bid,gvkey,permco,linktype,linkdt,linkenddt
0,10001,2015-01-02,11.02000,0.000000,8644.0,10.93000,10.90000,12994,7953.0,LC,1986-01-09,2017-08-31
1,10025,2015-01-02,58.00000,-0.002580,32084.0,58.32000,57.82000,11903,7975.0,LU,1986-01-30,2017-01-31
2,10026,2015-01-02,107.69000,-0.009929,88837.0,107.69000,107.45000,12825,7976.0,LU,1986-02-04,NaT
3,10028,2015-01-02,1.27000,0.040984,1500.0,1.26000,1.22000,12096,7978.0,LC,1986-02-05,NaT
4,10032,2015-01-02,40.57000,-0.015530,163298.0,40.57000,40.53000,12945,7980.0,LU,1986-02-05,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...
22958632,93426,2024-12-31,23.47000,0.010332,62710.0,23.53000,23.47000,185138,53443.0,LC,2010-07-07,NaT
22958633,93427,2024-12-31,219.88000,0.023841,416107.0,220.10001,219.74001,179583,53445.0,LC,2010-06-25,NaT
22958634,93429,2024-12-31,195.39999,-0.002705,471701.0,195.39999,195.38000,184500,53447.0,LC,2010-06-15,NaT
22958635,93434,2024-12-31,7.99000,-0.052195,12585.0,8.01000,7.73000,184259,53427.0,LC,2010-06-14,NaT


# Calculate return data for next day, next week and next month


In [ ]:
all_dfs = []
base_path = "/content/drive/My Drive/Thesis/mywork"

# Read Textual Data from 2015 - 2025
for year in range(2015, 2025):
    filename = f"{base_path}/regression_data_X_{year}_v3.parquet"
    df = pd.read_parquet(filename)
    df['year'] = year
    all_dfs.append(df)

# Merge all into one long DataFrame
merged_df = pd.concat(all_dfs, ignore_index=True)
merged_df

,transcriptid,keydevid,companyid,date,headline,Capital Structure,Company Performance,Competition & Challenge,ESG,Risk,...,Urgency and Emphasis_QA,gvkey,startdate,enddate,companyname,mostimportantdateutc,total_word_count,pre_word_count,qa_word_count,year
0,742893,279758760,296799,2015-01-05,"Piedmont Natural Gas Co. Inc., 2014 Earnings C...",0.128461,0.349422,0.199418,0.005011,-0.044203,...,NaN,8577,NaT,NaT,"Piedmont Natural Gas Company, Inc.",2015-01-05 15:00:00,4081,2259,1822,2015
1,743195,280545047,285016,2015-01-06,"Lindsay Corporation, Q1 2015 Earnings Call, Ja...",0.246220,-0.039146,-0.405990,0.498511,-0.001553,...,NaN,14954,NaT,NaT,Lindsay Corporation,2015-01-06 16:00:00,7387,1458,5929,2015
2,743314,279365378,262639,2015-01-06,"Commercial Metals Company, Q1 2015 Earnings Ca...",0.000366,-0.012593,0.015179,0.997591,0.006753,...,NaN,3246,NaT,NaT,Commercial Metals Company,2015-01-06 16:00:00,6661,1508,5153,2015
3,743360,279483592,289030,2015-01-06,"Micron Technology, Inc., Q1 2015 Earnings Call...",0.001424,0.489532,0.057839,0.000006,0.236014,...,NaN,7343,NaT,NaT,"Micron Technology, Inc.",2015-01-06 21:30:00,8613,5119,3494,2015
4,743374,280159116,188477,2015-01-06,"Sonic Corp., Q1 2015 Earnings Call, Jan 06, 2015",0.999999,0.648500,0.205575,0.988874,0.129439,...,NaN,23697,NaT,NaT,Sonic LLC,2015-01-06 22:00:00,10190,5854,4336,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119227,3373034,1869093659,203142,2024-02-06,"Arrowhead Pharmaceuticals, Inc., Q1 2024 Earni...",0.333226,0.301867,-0.006013,0.000530,0.281008,...,NaN,28806,2004-01-13,NaT,"Arrowhead Pharmaceuticals, Inc.",2024-02-06 21:30:00,9459,5558,3901,2024
119228,3373044,1871240813,34215227,2024-02-29,"Outbrain Inc., Q4 2023 Earnings Call, Feb 29, ...",0.999956,0.330431,0.362975,NaN,0.583406,...,NaN,39248,NaT,NaT,Outbrain Inc.,2024-02-29 13:30:00,7907,3824,4083,2024
119229,3375462,1897041907,137228789,2024-09-20,"Tamboran Resources Corporation, Q4 2024 Earnin...",0.371323,0.424922,NaN,0.186354,0.152466,...,NaN,348784,NaT,NaT,Tamboran Resources Corporation,2024-09-20 12:00:00,5151,2539,2612,2024
119230,3381531,1903003197,137228789,2024-11-12,"Tamboran Resources Corporation, Q1 2025 Earnin...",0.014821,0.557166,NaN,0.499927,0.520969,...,NaN,348784,NaT,NaT,Tamboran Resources Corporation,2024-11-12 13:00:00,3591,1911,1680,2024


In [ ]:
def compute_sentiment_returns(df_test_merge, df):

    # Change the date type
    df_test_merge['date'] = pd.to_datetime(df_test_merge['date'])
    df['date'] = pd.to_datetime(df['date'])

    # Sort values
    df_test_merge = df_test_merge.sort_values(['gvkey', 'date'])
    df = df.sort_values(['gvkey', 'date'])

    # Select all the sentiment signal columns
    sentiment_cols = [col for col in df.columns if col not in ['gvkey', 'date']]

    output = []

    for gvkey, df_earning in df.groupby('gvkey'):
        df_price = df_test_merge[df_test_merge['gvkey'] == gvkey].reset_index(drop=True)

        for _, row in df_earning.iterrows():
            event_date = row['date']

            # Find the Monday of the week containing event_date
            event_week_start = event_date - pd.Timedelta(days=event_date.weekday())
            rebalance_start = event_week_start + pd.Timedelta(days=7)  # Monday of the next week

           # Find the return for the full next week (5 trading days)
            week_data = df_price[
                (df_price['date'] >= rebalance_start) &
                (df_price['date'] < rebalance_start + pd.Timedelta(days=7))
            ]

            # Find the return for the full next month (approximately 20 trading days)
            month_data = df_price[
                (df_price['date'] >= rebalance_start) &
                (df_price['date'] < rebalance_start + pd.Timedelta(days=28))
            ]

            # Get next day's return
            next_day_data = df_price[df_price['date'] > event_date]
            next_day_data = next_day_data[next_day_data['date'] == next_day_data['date'].min()]
            next_day_return = next_day_data['ret'].values[0] if not next_day_data.empty else np.nan

            # Only keep periods with sufficient number of trading days
            weekly_return = (week_data['ret'] + 1).prod() - 1 if len(week_data) >= 3 else np.nan
            monthly_return = (month_data['ret'] + 1).prod() - 1 if len(month_data) >= 15 else np.nan

            entry = {
                'gvkey': gvkey,
                'date': event_date,
                'next_day_return': next_day_return,
                'weekly_return': weekly_return,
                'monthly_return': monthly_return,
            }

            # Add all sentiment columns
            for col in sentiment_cols:
                entry[col] = row[col]

            output.append(entry)

    return pd.DataFrame(output)


In [ ]:
# Extract the columns for sentiment analysis only
sentiment_cols = [
    'Capital Structure', 'Company Performance', 'Competition & Challenge',
       'ESG', 'Risk', 'Strategy and Operation', 'Technology', 'Trend',
       'Urgency and Emphasis', 'Capital Structure_Pre',
       'Company Performance_Pre', 'Competition & Challenge_Pre', 'ESG_Pre',
       'Risk_Pre', 'Strategy and Operation_Pre', 'Trend_Pre',
       'Urgency and Emphasis_Pre', 'Capital Structure_QA',
       'Company Performance_QA', 'Competition & Challenge_QA', 'ESG_QA',
       'Strategy and Operation_QA', 'Trend_QA', 'Risk_QA', 'Technology_QA',
       'Technology_Pre', 'Urgency and Emphasis_QA']
columns_to_keep = ['gvkey', 'date'] + sentiment_cols
df_filtered = merged_df[columns_to_keep]
df_filtered

,gvkey,date,Capital Structure,Company Performance,Competition & Challenge,ESG,Risk,Strategy and Operation,Technology,Trend,...,Capital Structure_QA,Company Performance_QA,Competition & Challenge_QA,ESG_QA,Strategy and Operation_QA,Trend_QA,Risk_QA,Technology_QA,Technology_Pre,Urgency and Emphasis_QA
0,8577,2015-01-05,0.128461,0.349422,0.199418,0.005011,-0.044203,0.079182,NaN,0.750172,...,-0.005978,0.116617,0.000004,-0.000029,0.004468,0.666899,NaN,NaN,NaN,NaN
1,14954,2015-01-06,0.246220,-0.039146,-0.405990,0.498511,-0.001553,0.194751,1.000000,-0.001048,...,0.339241,0.045234,-0.405990,0.498511,0.029013,-0.001934,-0.251940,1.000000e+00,NaN,NaN
2,3246,2015-01-06,0.000366,-0.012593,0.015179,0.997591,0.006753,0.014687,NaN,0.426144,...,NaN,-0.230717,-0.004743,0.997591,0.153275,0.204050,0.008158,NaN,NaN,NaN
3,7343,2015-01-06,0.001424,0.489532,0.057839,0.000006,0.236014,0.493374,0.800040,0.573369,...,NaN,0.295177,0.057839,NaN,0.072734,-0.276663,0.727041,NaN,0.800040,NaN
4,23697,2015-01-06,0.999999,0.648500,0.205575,0.988874,0.129439,0.484663,0.762843,0.543079,...,NaN,0.177161,-0.477569,NaN,0.023141,NaN,0.665749,NaN,0.762843,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119227,28806,2024-02-06,0.333226,0.301867,-0.006013,0.000530,0.281008,0.476302,0.285412,0.368912,...,-0.000161,0.002461,-0.006013,0.000530,-0.000131,0.486902,0.002186,-8.636489e-07,0.399577,NaN
119228,39248,2024-02-29,0.999956,0.330431,0.362975,NaN,0.583406,0.654538,0.993047,0.669306,...,NaN,0.206845,0.028847,NaN,NaN,0.009144,NaN,NaN,0.993047,NaN
119229,348784,2024-09-20,0.371323,0.424922,NaN,0.186354,0.152466,0.300848,0.000021,NaN,...,0.164565,-0.001785,NaN,0.014181,-0.000035,NaN,NaN,NaN,0.000021,NaN
119230,348784,2024-11-12,0.014821,0.557166,NaN,0.499927,0.520969,0.585666,0.666700,NaN,...,0.014821,NaN,NaN,-0.000146,0.999991,NaN,0.000303,NaN,0.666700,NaN


In [ ]:
# Get the return data and save it to a file
final_df = compute_sentiment_returns(df_test_merge, df_filtered)
output_path = f"/content/drive/My Drive/Thesis/mywork/sentiment_returns_all_years.parquet"
final_df.to_parquet(output_path, index=False)

<ipython-input-9-b0b4d0c4c656>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'])


In [ ]:
final_df

,gvkey,date,next_day_return,weekly_return,monthly_return,Capital Structure,Company Performance,Competition & Challenge,ESG,Risk,...,Capital Structure_QA,Company Performance_QA,Competition & Challenge_QA,ESG_QA,Strategy and Operation_QA,Trend_QA,Risk_QA,Technology_QA,Technology_Pre,Urgency and Emphasis_QA
0,1004,2015-03-30,-0.024158,-0.004576,-0.012006,0.166549,0.390272,0.499978,NaN,-0.127812,...,-0.000141,0.090313,0.999999,NaN,0.611526,0.599943,-0.315781,NaN,NaN,NaN
1,1004,2015-07-14,-0.043210,-0.077526,-0.108956,0.480665,0.146112,-0.026671,NaN,0.344811,...,NaN,0.272562,-0.026671,NaN,0.326759,1.000000,0.402183,NaN,NaN,NaN
2,1004,2015-09-24,-0.095215,0.073506,0.219991,-0.000061,0.223542,-0.046888,0.001573,-0.098396,...,-0.000230,0.300677,-0.046888,NaN,0.389017,0.249848,-0.122991,NaN,NaN,NaN
3,1004,2015-12-17,0.068606,0.080456,-0.030069,0.268410,0.324073,0.186048,-0.000023,0.391562,...,0.020225,0.999996,0.465299,-0.000023,-0.198682,0.997565,0.499726,NaN,NaN,NaN
4,1004,2016-03-22,0.049681,0.025877,0.060966,0.276640,0.652487,-0.000239,0.998155,0.186078,...,0.001974,0.324465,-0.000239,0.998155,-0.148496,0.206847,-0.464402,NaN,NaN,-0.003277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119227,349337,2023-11-14,-0.014545,-0.045801,0.061068,NaN,0.079446,NaN,NaN,0.037070,...,NaN,0.012824,NaN,NaN,NaN,NaN,NaN,0.869467,0.591287,NaN
119228,349337,2024-03-27,0.004717,0.117372,-0.131454,0.500326,0.238157,0.085643,NaN,0.164576,...,NaN,0.454525,NaN,NaN,0.020255,-0.000043,0.000392,0.213412,0.693661,NaN
119229,349337,2024-05-07,0.005814,0.023392,-0.093567,-0.278886,0.285979,-0.076892,0.994889,0.225685,...,-0.000473,-0.083236,NaN,0.989779,-0.001855,0.023601,0.008075,0.000067,0.488974,NaN
119230,349337,2024-08-12,-0.096386,-0.038710,-0.167743,0.143871,0.305791,-0.130532,NaN,0.211139,...,NaN,0.317151,NaN,NaN,0.227898,NaN,0.333315,-0.238407,0.701661,NaN
